In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, count

In [2]:
spark = SparkSession.builder.master('local[*]').appName('used-car-price').config('spark.executor.memory', '6g').getOrCreate()
spark = SparkSession.builder.master('local[*]').appName('used-car-price')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()

24/09/10 20:00:59 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/10 20:00:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/10 20:01:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/10 20:01:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/10 20:01:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/10 20:01:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
csv_file_path = '../data/cleaned_train.csv'
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

In [4]:
df.show()

+-----+-------------+--------------------+----------+------+-------------+--------------------+--------------------+--------------------+-------------+---------------+-----------+-----+
|   id|        brand|               model|model_year|milage|    fuel_type|              engine|        transmission|             ext_col|      int_col|       accident|clean_title|price|
+-----+-------------+--------------------+----------+------+-------------+--------------------+--------------------+--------------------+-------------+---------------+-----------+-----+
|57122|      Porsche|             911 GT3|      2017| 72500|     Gasoline|5020HP 40L Flat 6...|      6-speed manual|              Silver|        Black|  None reported|        Yes|27500|
|57123|         Jeep|Wrangler Unlimite...|      2022| 16184|     Gasoline|20L I4 16V GDI DO...|   8-speed automatic|                Gray|        Black|  None reported|       NULL|46599|
|57125|      Porsche|         911 Carrera|      2005|106385|     Gasol

In [5]:
data = df.drop('id')

In [6]:
data.show()

+-------------+--------------------+----------+------+-------------+--------------------+--------------------+--------------------+-------------+---------------+-----------+-----+
|        brand|               model|model_year|milage|    fuel_type|              engine|        transmission|             ext_col|      int_col|       accident|clean_title|price|
+-------------+--------------------+----------+------+-------------+--------------------+--------------------+--------------------+-------------+---------------+-----------+-----+
|      Porsche|             911 GT3|      2017| 72500|     Gasoline|5020HP 40L Flat 6...|      6-speed manual|              Silver|        Black|  None reported|        Yes|27500|
|         Jeep|Wrangler Unlimite...|      2022| 16184|     Gasoline|20L I4 16V GDI DO...|   8-speed automatic|                Gray|        Black|  None reported|       NULL|46599|
|      Porsche|         911 Carrera|      2005|106385|     Gasoline|3500HP 34L Flat 6...|      6-spe

In [7]:
from sklearn.impute import KNNImputer
import pandas as pd
def knn_impute(df, n_neighbors=5):   
    df_encoded = df.copy()
    for col in df_encoded.select_dtypes(include='object').columns:
        df_encoded[col] = df_encoded[col].astype('category').cat.codes
    knn_imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed = pd.DataFrame(knn_imputer.fit_transform(df_encoded), columns=df_encoded.columns)
    for col in df.select_dtypes(include='object').columns:
        df_imputed[col] = df_imputed[col].round().astype(int).map(
            dict(enumerate(df[col].astype('category').cat.categories)))
    return df_imputed

In [8]:
df_train_imputed = knn_impute(data.toPandas(), n_neighbors=25)

In [9]:
from sklearn.preprocessing import OrdinalEncoder
cat_cols_train = df_train_imputed.select_dtypes(include=['object']).columns
cat_cols_train = cat_cols_train[cat_cols_train != 'class']
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

df_train_imputed[cat_cols_train] = ordinal_encoder.fit_transform(df_train_imputed[cat_cols_train].astype(str))


In [15]:
df_train_imputed.isnull().sum()

brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

In [10]:
Sp_data = spark.createDataFrame(df_train_imputed)

In [11]:
Sp_data.show()

24/09/10 20:01:17 WARN TaskSetManager: Stage 5 contains a task of very large size (1119 KiB). The maximum recommended task size is 1000 KiB.


+-----+------+----------+--------+---------+------+------------+-------+-------+--------+-----------+-------+
|brand| model|model_year|  milage|fuel_type|engine|transmission|ext_col|int_col|accident|clean_title|  price|
+-----+------+----------+--------+---------+------+------------+-------+-------+--------+-----------+-------+
| 43.0| 157.0|    2017.0| 72500.0|      2.0| 952.0|        14.0|  262.0|   14.0|     0.0|        0.0|27500.0|
| 22.0|1775.0|    2022.0| 16184.0|      2.0| 229.0|        19.0|  127.0|   14.0|     0.0|        1.0|46599.0|
| 43.0| 143.0|    2005.0|106385.0|      2.0| 679.0|        14.0|   29.0|   14.0|     0.0|        0.0| 5000.0|
| 39.0| 915.0|    2010.0|137483.0|      2.0| 163.0|        24.0|  127.0|   14.0|     0.0|        0.0| 9950.0|
|  8.0| 603.0|    2017.0| 92800.0|      2.0| 855.0|        11.0|  127.0|   10.0|     1.0|        0.0|22550.0|
|  0.0|1302.0|    2011.0|156193.0|      2.0| 523.0|        24.0|  262.0|   14.0|     1.0|        0.0|30000.0|
| 19.0|151

In [12]:
Sp_data.columns

['brand',
 'model',
 'model_year',
 'milage',
 'fuel_type',
 'engine',
 'transmission',
 'ext_col',
 'int_col',
 'accident',
 'clean_title',
 'price']

In [14]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col

# Define feature columns excluding the target column (price)
feature_columns = [column for column in Sp_data.columns if column != "price"]

# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_df = assembler.transform(Sp_data)

# Standardize features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(assembled_df)
scaled_df = scaler_model.transform(assembled_df)

# Convert the 'scaled_features' vector to an array for easier handling
scaled_df = scaled_df.withColumn("scaled_features_array", vector_to_array(scaled_df["scaled_features"]))

# Get the number of features
num_features = len(feature_columns)

# Create individual columns for each feature in the array
for i in range(num_features):
    scaled_df = scaled_df.withColumn(f"scaled_feature_{i+1}", col("scaled_features_array")[i])

# Select the individual scaled features for saving
scaled_df_to_save = scaled_df.select([f"scaled_feature_{i+1}" for i in range(num_features)])

# Save the scaled data to CSV
scaled_df_to_save.write.csv("../data/scaled_training_data.csv", header=True)

# Save the pre-fitted scaler model
scaler_model.write().overwrite().save("scaler_model")


24/09/10 20:03:12 WARN TaskSetManager: Stage 9 contains a task of very large size (1119 KiB). The maximum recommended task size is 1000 KiB.
24/09/10 20:03:14 WARN TaskSetManager: Stage 12 contains a task of very large size (1119 KiB). The maximum recommended task size is 1000 KiB.
